In [2]:
import requests
from starlette.requests import Request
from typing import Dict

from transformers import pipeline

from ray import serve

In [3]:
# Wrap the pretrained sentiment analysis model in a Serve deployment.

@serve.deployment

class SentimentAnalysisDeployment:

    def __init__(self):
        self._model = pipeline("sentiment-analysis")

    def __call__(self, request: Request) -> Dict:
        return self._model(request.query_params["text"])[0]
        

In [4]:
# Deploy the deployment.

serve.run(SentimentAnalysisDeployment.bind(), route_prefix="/")

2024-05-24 15:57:43,596	INFO worker.py:1740 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
(ProxyActor pid=61342) INFO 2024-05-24 15:57:46,974 proxy 127.0.0.1 proxy.py:1161 - Proxy starting on node 412c6eee99f29d8d7e77b6525f0b7137684fcf09578a8995877b2709 (HTTP port: 8000).
2024-05-24 15:57:47,009	INFO handle.py:126 -- Created DeploymentHandle 'pkxnsi5q' for Deployment(name='SentimentAnalysisDeployment', app='default').
2024-05-24 15:57:47,010	INFO handle.py:126 -- Created DeploymentHandle 'ww0fw083' for Deployment(name='SentimentAnalysisDeployment', app='default').
(ServeController pid=61341) INFO 2024-05-24 15:57:47,112 controller 61341 deployment_state.py:1598 - Deploying new version of Deployment(name='SentimentAnalysisDeployment', app='default') (initial target replicas: 1).
(ServeController pid=61341) INFO 2024-05-24 15:57:47,215 controller 61341 deployment_state.py:1844 - Adding 1 replica to Deployment(name='SentimentAnalysisDeployment', app='default').
(S

DeploymentHandle(deployment='SentimentAnalysisDeployment')

In [6]:
# 3: Query the deployment and print the result.

print(
    requests.get(
        "http://localhost:8000/", params={"text": "Ray Serve is great!"}
    ).json()
)


{'label': 'POSITIVE', 'score': 0.9998476505279541}


(ServeReplica:default:SentimentAnalysisDeployment pid=61343) INFO 2024-05-24 16:01:21,237 default_SentimentAnalysisDeployment 8wcolii6 cac1bd2a-45a5-4fc2-bd94-4a37b2f53e8a / replica.py:373 - __CALL__ OK 64.3ms
